In [45]:
import subprocess
import random
import string
import fuzzing

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import base64
import pandas as pd

# Defining the parameters for running the fuzzig code
num_iterations = 100
max_input_length_int = 5
number_of_fuzzed_variants_to_generate = 10
fuzz_factor = 7
min_input_length_string = 30
max_input_length_string = 50


columns = ['Row Number', 'Random Input', 'Fuzzed Input', 'Encrypted Fuzzed Input', 'Decrypted Fuzzed Input', 'Result']
# defining a dataframe to capture the results and the input data
fuzz_test_results = pd.DataFrame(columns=columns)
row_number = 1

# generate random inputs then fuzz the inputs and run them through the encryption and decryption functions
for i in range(num_iterations):
    # generate a random input for the key
    input_int = ''.join(random.choice(string.digits + string.digits) for _ in range(random.randint(1, max_input_length_int)))
    
    # Generating a random string value -- Data to Encrypt
    file_to_encrypt_data = ''.join(random.choice(string.ascii_lowercase+string.ascii_uppercase ) for _ in range(random.randint(min_input_length_int_string, max_input_length_int_string)))
    
    # Using the fuzzing package to create a list of fuzzed inputs which was generated above in file_to_encrypt_data variable
    file_to_encrypt_fdata = fuzzing.fuzz_string(file_to_encrypt_data, number_of_fuzzed_variants_to_generate, fuzz_factor)

    
    # code from the previous project for encryption and decryption -- ALice code and Bob Code
    b_bytes = int(input_int).to_bytes(4, 'big')
    
    password = b_bytes
    salt = b_bytes
    kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256,
            length=32,
            salt=salt,
            iterations=100000,
            backend=default_backend()
        )

    key = base64.urlsafe_b64encode(kdf.derive(password))
    f = Fernet(key)

    for j in file_to_encrypt_fdata:
        #print("fuzzed input" , j)
        fuzz_bytes = j.encode('utf-8')
        
        encrypted_data = f.encrypt(fuzz_bytes)
        
        decrypted_data_b = f.decrypt(encrypted_data)
        decrypted_data = decrypted_data_b.decode('utf-8')
        #print("fuzzed input bytes" , fuzz_bytes)
        
        #print("decrypted_data",decrypted_data)
        
        # Comparing the fuzzed input and the decrypted fuzzed data to check the encryption and decryption 
        if j == decrypted_data:
            row_to_insert = {'Row Number':row_number,'Random Input': file_to_encrypt_data, 'Fuzzed Input': j, 'Encrypted Fuzzed Input': encrypted_data, 'Decrypted Fuzzed Input': decrypted_data, 'Result': 'SUCCESS'}
            fuzz_test_results.loc[row_number] = row_to_insert
            row_number = row_number + 1
            
        else:
            #print("Error: Input and output do not match------------------------------------------------!")
            row_to_insert = {'Row Number':row_number,'Random Input': file_to_encrypt_data, 'Fuzzed Input': j, 'Encrypted Fuzzed Input': encrypted_data, 'Decrypted Fuzzed Input': decrypted_data, 'Result': 'FAIL'}
            fuzz_test_results.loc[row_number] = row_to_insert
            row_number = row_number + 1


fuzz_test_results



,Row Number,Random Input,Fuzzed Input,Encrypted Fuzzed Input,Decrypted Fuzzed Input,Result
1,1,dBLHSTcaaCbrtTFVDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,dBLHSTcaaCbrtTFÉDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,b'gAAAAABkVEiteU-jfhTzltWct-FpbWDl0jcU32YskrTI...,dBLHSTcaaCbrtTFÉDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,SUCCESS
2,2,dBLHSTcaaCbrtTFVDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,dB@HSTcaaCbrtTFVDoSeETPLCCyCALuACcxlvzSorzneC7O,b'gAAAAABkVEitAvkKsgGoBsFCQBinrRylRDLgC_7IO18g...,dB@HSTcaaCbrtTFVDoSeETPLCCyCALuACcxlvzSorzneC7O,SUCCESS
3,3,dBLHSTcaaCbrtTFVDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,dBLHSTcaaCbrtTFVDoSeTPLCCyLCALuACcxlvzSoBrzneCVO,b'gAAAAABkVEitE26iv-PV_dM4TJ2mkIpjpRSq1WJ3XuyR...,dBLHSTcaaCbrtTFVDoSeTPLCCyLCALuACcxlvzSoBrzneCVO,SUCCESS
4,4,dBLHSTcaaCbrtTFVDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,dBLHSTcaaCbrtTFVDoSeìTPLCCyLCALuACcxlvzSoBrzneCVO,b'gAAAAABkVEitI37-wKYrYGQ6w_OU60xx9cWCB9TUpWE5...,dBLHSTcaaCbrtTFVDoSeìTPLCCyLCALuACcxlvzSoBrzneCVO,SUCCESS
5,5,dBLHSTcaaCbrtTFVDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,dBLHSTcaaCbrtæFVDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,b'gAAAAABkVEitIf5bOvZu8OHmeg5TUna8DP28kNqYHriL...,dBLHSTcaaCbrtæFVDoSeETPLCCyLCALuACcxlvzSoBrzneCVO,SUCCESS
...,...,...,...,...,...,...
996,996,YkHcAeDnJrVtoEAmagWrwRhpcFvhlmhsSwX,YkHcAeDnJrVtoAmagWrwRhpcFvhlmhsSIX,b'gAAAAABkVEi2oI0Ftos5FFsVxh-C5JuuX7irAnK4Vqik...,YkHcAeDnJrVtoAmagWrwRhpcFvhlmhsSIX,SUCCESS
997,997,YkHcAeDnJrVtoEAmagWrwRhpcFvhlmhsSwX,YkHcAeDnJrVtoEAmagWrRhpcFhlmhsSwX,b'gAAAAABkVEi20ZZvKcvN4B6tc9A9tY4eMtKXAoW8Fggg...,YkHcAeDnJrVtoEAmagWrRhpcFhlmhsSwX,SUCCESS
998,998,YkHcAeDnJrVtoEAmagWrwRhpcFvhlmhsSwX,YkHcAeDnJrVtoEAmagUrwRhpcFvhlmhs!wX,b'gAAAAABkVEi206fEoNtYek1d7JV1n-7UtlllSWmCgw7u...,YkHcAeDnJrVtoEAmagUrwRhpcFvhlmhs!wX,SUCCESS
999,999,YkHcAeDnJrVtoEAmagWrwRhpcFvhlmhsSwX,YkHcAeDnJ¢VtoEAmagWrwR#pcFvlchsSwX,b'gAAAAABkVEi23R2G2VhYHzx9epcSOvGoDwZAKmAakjI4...,YkHcAeDnJ¢VtoEAmagWrwR#pcFvlchsSwX,SUCCESS


In [46]:
# Checking for the fail results from the results dataframe 

filtered_df = fuzz_test_results[fuzz_test_results['Result'] == 'FAIL']
filtered_df

,Row Number,Random Input,Fuzzed Input,Encrypted Fuzzed Input,Decrypted Fuzzed Input,Result


In [47]:
# Displaying the code to run a simple input message and display the different fuzzed varients of the input message.

import fuzzing
seed = "This is the content to be encrypted"
number_of_fuzzed_variants_to_generate = 10
fuzz_factor = 7
fuzzed_data = fuzzing.fuzz_string(seed, number_of_fuzzed_variants_to_generate, fuzz_factor)
#print(fuzzed_data)
row_number = 1
input_int = 123
b_bytes = int(input_int).to_bytes(4, 'big')
    
password = b_bytes
salt = b_bytes
kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256,
            length=32,
            salt=salt,
            iterations=100000,
            backend=default_backend()
        )

key = base64.urlsafe_b64encode(kdf.derive(password))
f = Fernet(key)


columns = ['Row Number', 'Input', 'Fuzzed Input', 'Decrypted Fuzzed Input', 'Result']
fuzz_test_sample = pd.DataFrame(columns=columns)

for i in fuzzed_data:
    fuzz_bytes = i.encode('utf-8')
        
    encrypted_data = f.encrypt(fuzz_bytes)
        
    decrypted_data_b = f.decrypt(encrypted_data)
    decrypted_data = decrypted_data_b.decode('utf-8')
        
    if i == decrypted_data:
        row_to_insert = {'Row Number':row_number,'Input': seed, 'Fuzzed Input': i, 'Decrypted Fuzzed Input': decrypted_data, 'Result': 'SUCCESS'}
        fuzz_test_sample.loc[row_number] = row_to_insert
        row_number = row_number + 1
            
    else:
        row_to_insert = {'Row Number':row_number,'Input': seed, 'Fuzzed Input': i, 'Decrypted Fuzzed Input': decrypted_data, 'Result': 'FAIL'}
        fuzz_test_sample.loc[row_number] = row_to_insert
        row_number = row_number + 1
    
fuzz_test_sample

,Row Number,Input,Fuzzed Input,Decrypted Fuzzed Input,Result
1,1,This is the content to be encrypted,This ij the content t b¿ encêypted,This ij the content t b¿ encêypted,SUCCESS
2,2,This is the content to be encrypted,ThisT»sthe cºntent toÄbe encrypted,ThisT»sthe cºntent toÄbe encrypted,SUCCESS
3,3,This is the content to be encrypted,ThisVis he content to be encrypte,ThisVis he content to be encrypte,SUCCESS
4,4,This is the content to be encrypted,This is the conte t to be \tncvypted,This is the conte t to be \tncvypted,SUCCESS
5,5,This is the content to be encrypted,This is the content toÚbe encrypteÆ,This is the content toÚbe encrypteÆ,SUCCESS
6,6,This is the content to be encrypted,ThiÔ is the content to b¿ encrypted,ThiÔ is the content to b¿ encrypted,SUCCESS
7,7,This is the content to be encrypted,This3s the content to be encrypted,This3s the content to be encrypted,SUCCESS
8,8,This is the content to be encrypted,This is the Ñontent to be encr§pted,This is the Ñontent to be encr§pted,SUCCESS
9,9,This is the content to be encrypted,This is §h¢ contenj³to be excrypted,This is §h¢ contenj³to be excrypted,SUCCESS
10,10,This is the content to be encrypted,ThiY is the cnten to be encryptId,ThiY is the cnten to be encryptId,SUCCESS


In [48]:
# Displaying the code to run with multiple fuzz factors using a for loop and the code from the block 1 in this notebook.

import subprocess
import random
import string
import fuzzing

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.fernet import Fernet
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
import base64
import pandas as pd


# set the number of iterations and the maximum length of the input
num_iterations = 100
max_input_length_int = 5
number_of_fuzzed_variants_to_generate = 10
min_input_length_string = 30
max_input_length_string = 50

columns = ['Row Number', 'Random Input', 'Fuzzed Input', 'Encrypted Fuzzed Input', 'Decrypted Fuzzed Input', 'Result']

fuzz_test_results = pd.DataFrame(columns=columns)
row_number = 1

fuzzfactor

columns_ff = ['Row Number', 'Fuzz Factor', 'Number of errors']
fuzz_factor_results = pd.DataFrame(columns=columns_ff)
row_number_ff = 1


for fuzzfactor in range(10,20):
    fuzz_factor = fuzzfactor

    # generate random inputs and run them through the encryption and decryption functions
    for i in range(num_iterations):
        # generate a random input string
        input_int = ''.join(random.choice(string.digits + string.digits) for _ in range(random.randint(1, max_input_length_int)))

        file_to_encrypt_data = ''.join(random.choice(string.ascii_lowercase+string.ascii_uppercase ) for _ in range(random.randint(min_input_length_int_string, max_input_length_int_string)))
        #print("\n Random string input: ", file_to_encrypt_data)

        file_to_encrypt_fdata = fuzzing.fuzz_string(file_to_encrypt_data, number_of_fuzzed_variants_to_generate, fuzz_factor)
        #print("\n fuzzed input: ", file_to_encrypt_fdata)

        #print(input_int)
        #Converting the above generated int symmetric key to binary

        b_bytes = int(input_int).to_bytes(4, 'big')

        password = b_bytes
        salt = b_bytes
        kdf = PBKDF2HMAC(
                algorithm=hashes.SHA256,
                length=32,
                salt=salt,
                iterations=100000,
                backend=default_backend()
            )

        key = base64.urlsafe_b64encode(kdf.derive(password))
        f = Fernet(key)

        for j in file_to_encrypt_fdata:
            #print("fuzzed input" , j)
            fuzz_bytes = j.encode('utf-8')

            encrypted_data = f.encrypt(fuzz_bytes)

            decrypted_data_b = f.decrypt(encrypted_data)
            decrypted_data = decrypted_data_b.decode('utf-8')
            #print("fuzzed input bytes" , fuzz_bytes)

            #print("decrypted_data",decrypted_data)


            if j == decrypted_data:
                #continue
                #print("Success !!!")
                row_to_insert = {'Row Number':row_number,'Random Input': file_to_encrypt_data, 'Fuzzed Input': j, 'Encrypted Fuzzed Input': encrypted_data, 'Decrypted Fuzzed Input': decrypted_data, 'Result': 'SUCCESS'}
                fuzz_test_results.loc[row_number] = row_to_insert
                row_number = row_number + 1

            else:
                #print("Error: Input and output do not match------------------------------------------------!")
                row_to_insert = {'Row Number':row_number,'Random Input': file_to_encrypt_data, 'Fuzzed Input': j, 'Encrypted Fuzzed Input': encrypted_data, 'Decrypted Fuzzed Input': decrypted_data, 'Result': 'FAIL'}
                fuzz_test_results.loc[row_number] = row_to_insert
                row_number = row_number + 1
    

    
    fuzz_filtered_df = fuzz_test_results[fuzz_test_results['Result'] == 'FAIL']
    error_count = len(fuzz_filtered_df)
    
    
    row_to_insert_f = {'Row Number':row_number_ff,'Fuzz Factor': fuzz_factor, 'Number of errors': error_count}
    fuzz_factor_results.loc[row_number_ff] = row_to_insert_f
    row_number_ff = row_number_ff + 1


fuzz_test_results

,Row Number,Random Input,Fuzzed Input,Encrypted Fuzzed Input,Decrypted Fuzzed Input,Result
1,1,MhvVXEFlixJHrbmfLIvpgRgjBnnjaJywgVKMLaJMfLatE,MhvVXEFlixJHrbmfLIvpgRgjBnnjaJywgVKMLaJÁfLatE,b'gAAAAABkVEjAdahYx4xzsN_MARt86PPGiySCsTQ0wtgR...,MhvVXEFlixJHrbmfLIvpgRgjBnnjaJywgVKMLaJÁfLatE,SUCCESS
2,2,MhvVXEFlixJHrbmfLIvpgRgjBnnjaJywgVKMLaJMfLatE,MvVXEFlixJHrbÓfLIvpgRgjBnnja?ywgVKMLaJMfLatE,b'gAAAAABkVEjAFmGm07YXbtJ9DwSG-FNTmYY1VJwig-Sv...,MvVXEFlixJHrbÓfLIvpgRgjBnnja?ywgVKMLaJMfLatE,SUCCESS
3,3,MhvVXEFlixJHrbmfLIvpgRgjBnnjaJywgVKMLaJMfLatE,MhvVXEFlixJHrbmfLIv»gRgjBnnjaJywgVKMLaJMfLatE,b'gAAAAABkVEjAmMXO4XBk8j6WzVnYab5rTLAOYdPEQNX6...,MhvVXEFlixJHrbmfLIv»gRgjBnnjaJywgVKMLaJMfLatE,SUCCESS
4,4,MhvVXEFlixJHrbmfLIvpgRgjBnnjaJywgVKMLaJMfLatE,MhvVXEFlixJHrbfLIvpggjBnnjaJywgVKMaJMfLatE,b'gAAAAABkVEjAIYAkEu91AiEmpSI8m68pN6UGnZVk_p9s...,MhvVXEFlixJHrbfLIvpggjBnnjaJywgVKMaJMfLatE,SUCCESS
5,5,MhvVXEFlixJHrbmfLIvpgRgjBnnjaJywgVKMLaJMfLatE,MhvDXEFlixJHrbmfLIvpgRgjBnnjaJywVÉMLaJMfLatE,b'gAAAAABkVEjAb17px5mR_18xjrdJl6Q22I0_4xDgFOMQ...,MhvDXEFlixJHrbmfLIvpgRgjBnnjaJywVÉMLaJMfLatE,SUCCESS
...,...,...,...,...,...,...
9996,9996,DDaAXpWXarhwqHiOrgAFULhlxMuzaxFIKwGWcUJrASsMBK...,DDaAXpWXarhwqHiOrgAFULhlxMuzaxFIKwGWcUJrASsMBK...,b'gAAAAABkVEkdsnHEnJwcNhdmj7bopQJCTatbQluDBWvw...,DDaAXpWXarhwqHiOrgAFULhlxMuzaxFIKwGWcUJrASsMBK...,SUCCESS
9997,9997,DDaAXpWXarhwqHiOrgAFULhlxMuzaxFIKwGWcUJrASsMBK...,DDaAXpWXarhwqHiOrgAFULhlxMuza¦FIKwGWcUJrA$sMjK...,b'gAAAAABkVEkdgyXLYLRTk1eU2oPdZkIw6gX2P_k9FIDe...,DDaAXpWXarhwqHiOrgAFULhlxMuza¦FIKwGWcUJrA$sMjK...,SUCCESS
9998,9998,DDaAXpWXarhwqHiOrgAFULhlxMuzaxFIKwGWcUJrASsMBK...,åD`AXpWXarhwqHiOrAFULhlxMuzaxFIKwGWcUJrASsMBK...,b'gAAAAABkVEkdKyAUFbOY6iw19xzDZ652_h3SVIURVcxi...,åD`AXpWXarhwqHiOrAFULhlxMuzaxFIKwGWcUJrASsMBK...,SUCCESS
9999,9999,DDaAXpWXarhwqHiOrgAFULhlxMuzaxFIKwGWcUJrASsMBK...,DDaAXpWXar_wqHiOrgAFULhlxMuzaxFIwGWcUJrASsMBK...,b'gAAAAABkVEkdDSTxTLfb4dZRKEQOKzpX9u1U6kHWLwbQ...,DDaAXpWXar_wqHiOrgAFULhlxMuzaxFIwGWcUJrASsMBK...,SUCCESS


In [49]:
fuzz_factor_results

,Row Number,Fuzz Factor,Number of errors
1,1,10,0
2,2,11,0
3,3,12,0
4,4,13,0
5,5,14,0
6,6,15,0
7,7,16,0
8,8,17,0
9,9,18,0
10,10,19,0
